<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [1]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [2]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

#количества состояний/входов/выходов
elem = 100
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [3]:
def x1(x, v):
  return math.sin(math.log(abs(v[1] + v[0])) / x[2])

def x2(x, v):
  return math.cos(v[0] * x[2] - (x[1] / x[2])**3)

def x3(x, v):
  return math.sin(math.tan(v[0] / v[2]) - 3**(x[2]))

def v1(x, v):
  return (x[1] * v[0] * v[1] + math.tan(x[2])**2)

def v2(x, v):
  return (0.2 * v[0] + 33.0 * v[1] + v[2])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * x[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0] / x[1]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[2])))

def y3(x, v):
  return v[2] * v[0] / v[0]

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [4]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps = 0.000001): #А - np.array, eps - точность вычислительная
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#решение СЛАУ вида AX=B
def solvingSLAE(A, B):
  A = np.array(A.copy())

  #проверка на бесконечное количество решений
  A_psev = BenIsrael(A)

  X = A_psev.dot(B)
  return X

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#Создание ДОМ в определённый момент времени
def createTableDNM(functions, count, step, oldTable = []): #count(x, v, y)
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

#заполнение
def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [5]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDNM(functions, count, 0)
model_0

,x1(0),x2(0),x3(0),v1(0),v2(0),v3(0),y1(0),y2(0),y3(0)
0,0.846132,0.935183,0.961528,-0.954028,-93.675560,1.318872,-0.843622,0.984091,1.318872
1,0.613628,0.785410,0.818524,3.458713,-65.236534,1.009485,0.156818,0.920086,1.009485
2,0.455045,-0.470659,0.920380,-6.859849,42.279868,-0.129874,-0.065225,0.359874,-0.129874
3,0.829441,-0.564311,0.085281,2.545118,-39.678371,0.428361,0.395104,0.644500,0.428361
4,0.239530,-0.170451,-0.984080,23.307232,-16.058441,3.299286,0.012183,0.396284,3.299286
...,...,...,...,...,...,...,...,...,...
95,-0.366811,0.956279,-0.998479,0.373503,72.225237,0.925929,-0.608242,0.893968,0.925929
96,0.646414,-0.557787,-0.103728,9.907029,-2.292034,0.761186,0.070241,0.830530,0.761186
97,-0.129035,0.615978,-0.316611,-1.266849,-88.800157,2.526096,0.046618,0.759845,2.526096
98,-0.118010,0.587784,0.975379,-7.470659,-40.967493,1.450782,0.007078,0.996397,1.450782


In [6]:
model_1 = createTableDNM(functions, count, 1, np.array(model_0))
model_1

,x1(1),x2(1),x3(1),v1(1),v2(1),v3(1),y1(1),y2(1),y3(1)
0,-0.999807,-0.263416,0.578830,14.058154,16.744298,1.637527,-0.138556,0.998887,1.637527
1,-0.947525,-0.367913,-0.828500,2.790100,-14.792527,11.865296,-0.544996,0.803143,11.865296
2,-0.670054,0.994675,0.270356,-8.395663,-94.982897,10.995094,0.065504,1.000000,10.995094
3,-0.999611,0.602123,-0.993167,15.520315,-72.115683,3.629638,-0.082986,0.684763,3.629638
4,-0.903862,-0.581498,0.607020,-10.830985,34.878101,5.444788,0.104043,0.862306,5.444788
...,...,...,...,...,...,...,...,...,...
95,0.912715,0.874906,0.092761,1.789065,-47.418502,12.394981,0.521070,0.412979,12.394981
96,-0.660864,0.848688,-0.399183,35.238076,79.993114,7.696006,-0.016549,0.993754,7.696006
97,-0.996990,0.088691,-0.950386,-1.858424,9.541224,0.553671,1.798670,0.725130,0.553671
98,-0.742397,0.341406,-0.702740,58.952450,87.758477,0.840824,-0.018570,0.863245,0.840824


In [7]:
model_2 = createTableDNM(functions, count, 2, np.array(model_1))
model_2

,x1(2),x2(2),x3(2),v1(2),v2(2),v3(2),y1(2),y2(2),y3(2)
0,-0.353768,-0.368644,-0.137267,8.533387,0.306110,0.216017,-0.040612,0.462970,0.216017
1,-0.141589,-0.736833,-0.162144,4.944712,-31.998421,-0.687267,-0.012552,0.796509,-0.687267
2,-0.992551,-0.232088,-0.743615,-3.604993,27.758588,0.436852,0.569375,0.650453,0.436852
3,0.796873,-0.869535,0.971924,-7.778274,-62.696127,0.043933,-0.098075,0.209569,0.043933
4,-0.864651,0.832246,0.296024,-2.814814,-15.484239,2.020910,0.313102,0.948893,2.020910
...,...,...,...,...,...,...,...,...,...
95,-0.339463,-0.997038,-0.820299,21.077851,52.639626,18.894423,-0.009397,0.211784,18.894423
96,0.624692,-0.253232,0.488190,0.297876,50.674672,1.356238,3.293849,0.988469,1.356238
97,-0.839393,-0.194980,-0.539916,7.434949,78.941546,1.233960,-0.234248,0.971496,1.233960
98,-0.728004,-0.890263,0.885456,4.862411,-2.252085,-0.293415,-0.135391,0.537795,-0.293415


##1. Линейная форма функции пересчёта

###Для функции $x_1[t+1]$

In [8]:
#x3, v1, v2
vector_ones = [1] * elem
x1_linearFunc = model_0[['x3(0)', 'v1(0)', 'v2(0)']]
x1_linearFunc['x1(1)'] = model_1['x1(1)']
x1_linearFunc.insert(0, "optional_col", vector_ones)

A1_linear = np.array(x1_linearFunc)
#A1_psev = BenIsrael(A1_linear[:, 0:(len(A1_linear[0]) - 1)])
X1_linear = solvingSLAE(A1_linear[:, 0:(len(A1_linear[0]) - 1)], np.matrix(A1_linear[:, len(A1_linear[0]) - 1]).T)
print(pd.DataFrame(X1_linear, columns=['X1'], index=['a1', 'c13', 'b11', 'b12']))

           X1
a1   0.016095
c13 -0.349809
b11  0.000018
b12  0.002569


<ipython-input-8-e7a1fb8a3e1a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_linearFunc['x1(1)'] = model_1['x1(1)']


In [9]:
X1_estimate_linear = []
X1_variance_linear = []
X1_error_linear = []
for i in range(len(x1_linearFunc)):
  X1_estimate_linear.append(X1_linear[0,0] + X1_linear[1,0] * np.array(x1_linearFunc)[i,1]
                            + X1_linear[2,0] * np.array(x1_linearFunc)[i,2] + X1_linear[3,0] * np.array(x1_linearFunc)[i,3])

  X1_variance_linear.append((1.0 / len(x1_linearFunc)) * (np.array(x1_linearFunc)[i, len(np.array(x1_linearFunc)[0]) - 1] - X1_estimate_linear[i])**2)

  X1_error_linear.append((1.0 / len(x1_linearFunc)) * abs((np.array(x1_linearFunc)[i, len(np.array(x1_linearFunc)[0]) - 1] - X1_estimate_linear[i])
                            / np.array(x1_linearFunc)[i, len(np.array(x1_linearFunc)[0]) - 1]) * 100)

x1_linearFunc['X1*'] = X1_estimate_linear
x1_linearFunc['(X1 - X1*)^2'] = X1_variance_linear
x1_linearFunc['|(X1 - X1*)| / |X1|, %'] = X1_error_linear
x1_linearFunc

<ipython-input-9-0899adfbda21>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_linearFunc['X1*'] = X1_estimate_linear
<ipython-input-9-0899adfbda21>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_linearFunc['(X1 - X1*)^2'] = X1_variance_linear
<ipython-input-9-0899adfbda21>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

,optional_col,x3(0),v1(0),v2(0),x1(1),X1*,(X1 - X1*)^2,"|(X1 - X1*)| / |X1|, %"
0,1,0.961528,-0.954028,-93.675560,-0.999807,-0.560965,0.001926,0.438927
1,1,0.818524,3.458713,-65.236534,-0.947525,-0.437790,0.002598,0.537964
2,1,0.920380,-6.859849,42.279868,-0.670054,-0.197349,0.002234,0.705473
3,1,0.085281,2.545118,-39.678371,-0.999611,-0.115642,0.007814,0.884313
4,1,-0.984080,23.307232,-16.058441,-0.903862,0.319490,0.014966,1.353472
...,...,...,...,...,...,...,...,...
95,1,-0.998479,0.373503,72.225237,0.912715,0.550957,0.001309,0.396354
96,1,-0.103728,9.907029,-2.292034,-0.660864,0.046668,0.005006,1.070616
97,1,-0.316611,-1.266849,-88.800157,-0.996990,-0.101339,0.008022,0.898355
98,1,0.975379,-7.470659,-40.967493,-0.742397,-0.430497,0.000973,0.420126


###Для функции $x_2[t+1]$

In [20]:
#x2, x3, v1
x2_linear = model_0[['x2(0)', 'x3(0)', 'v1(0)']]
x2_linear['x2(1)'] = model_1['x2(1)']
x2_linear.insert(0, "optional_col", vector_ones)

A2_linear = np.array(x2_linear)
#A2_psev = BenIsrael(A2_linear[:, 0:(len(A2_linear[0]) - 1)])
X2_estim_linear = solvingSLAE(A2_linear[:, 0:(len(A2_linear[0]) - 1)], np.matrix(A2_linear[:, len(A2_linear[0]) - 1]).T)
print(pd.DataFrame(X2_estim_linear, columns=['X2'], index=['a2', 'c22', 'c23', 'b21']))

           X2
a2   0.051713
c22  0.248534
c23 -0.111496
b21 -0.000021


<ipython-input-20-d13e8e2c6e45>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x2_linear['x2(1)'] = model_1['x2(1)']


           X2
a2   0.051713
c22  0.248534
c23 -0.111496
b21 -0.000021


###Для функции $x_3[t+1]$

In [21]:
#x3, v1, v3
x3_linear = model_0[['x3(0)', 'v1(0)', 'v3(0)']]
x3_linear['x3(1)'] = model_1['x3(1)']
x3_linear.insert(0, "optional_col", vector_ones)

A3_linear = np.array(x3_linear)
#A3_psev = BenIsrael(A3_linear[:, 0:(len(A3_linear[0]) - 1)])
X3_estim_linear = solvingSLAE(A3_linear[:, 0:(len(A3_linear[0]) - 1)], np.matrix(A3_linear[:, len(A3_linear[0]) - 1]).T)
print(pd.DataFrame(X3_estim_linear, columns=['X3'], index=['a3', 'c33', 'b31', 'b33']))

           X3
a3  -0.148892
c33 -0.059680
b31  0.000024
b33 -0.033668


<ipython-input-21-39d182782a9e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x3_linear['x3(1)'] = model_1['x3(1)']


###Для функции $y_1[t]$

In [22]:
#x1, x2, v1
y1_linear = model_0[['x1(0)', 'x2(0)', 'v1(0)']]
y1_linear['y1(0)'] = model_0['y1(0)']
y1_linear.insert(0, "optional_col", vector_ones)

A1y_linear = np.array(y1_linear)
#A1y_psev = BenIsrael(A1y_linear[:, 0:(len(A1y_linear[0]) - 1)])
Y1_estim_linear = solvingSLAE(A1y_linear[:, 0:(len(A1y_linear[0]) - 1)], np.matrix(A1y_linear[:, len(A1y_linear[0]) - 1]).T)
print(pd.DataFrame(Y1_estim_linear, columns=['Y1'], index=['a1', 'c11', 'c12', 'b11']))

            Y1
a1   -9.911855
c11 -25.824722
c12 -11.821935
b11   0.000194


<ipython-input-22-8f669e901372>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1_linear['y1(0)'] = model_0['y1(0)']


###Для функции $y_2[t]$

In [23]:
#v3
y2_linear = model_0[['v3(0)']]
y2_linear['y2(0)'] = model_0['y2(0)']
y2_linear.insert(0, "optional_col", vector_ones)

A2y_linear = np.array(y2_linear)
#A2y_psev = BenIsrael(A2y_linear[:, 0:(len(A2y_linear[0]) - 1)])
Y2_estim_linear = solvingSLAE(A2y_linear[:, 0:(len(A2y_linear[0]) - 1)], np.matrix(A2y_linear[:, len(A2y_linear[0]) - 1]).T)
print(pd.DataFrame(Y2_estim_linear, columns=['Y2'], index=['a2', 'b23']))

           Y2
a2   0.752667
b23 -0.002954


<ipython-input-23-3f752691fe1c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y2_linear['y2(0)'] = model_0['y2(0)']


###Для функции $y_3[t]$

In [24]:
#v1, v3
y3_linear = model_0[['v1(0)', 'v3(0)']]
y3_linear['y3(0)'] = model_0['y3(0)']
y3_linear.insert(0, "optional_col", vector_ones)

A3y_linear = np.array(y3_linear)
#A3y_psev = BenIsrael(A3y_linear[:, 0:(len(A3y_linear[0]) - 1)])
Y3_estim_linear = solvingSLAE(A3y_linear[:, 0:(len(A3y_linear[0]) - 1)], np.matrix(A3y_linear[:, len(A3y_linear[0]) - 1]).T)
print(pd.DataFrame(Y3_estim_linear, columns=['Y3'], index=['a3', 'b31', 'b33']))

               Y3
a3   4.085066e-13
b31 -1.238701e-17
b33  1.000000e+00


<ipython-input-24-cb33e6054e94>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y3_linear['y3(0)'] = model_0['y3(0)']
